# 5 diferenças entre Random Forest e AdaBoost:

- Método de ensemble: Random Forest usa bagging (bootstrap aggregating), criando árvores independentes em paralelo com amostras aleatórias dos dados. AdaBoost usa boosting, construindo árvores sequencialmente onde cada nova árvore foca nos erros das anteriores.

- Ponderação de amostras: Random Forest trata todas as amostras com igual importância durante o treinamento. AdaBoost atribui pesos maiores às amostras classificadas incorretamente, forçando o modelo a focar nos exemplos difíceis.

- Complexidade das árvores: Random Forest geralmente usa árvores profundas e completas. AdaBoost usa "árvores fracas" (weak learners), frequentemente árvores de decisão com apenas um nível (stumps) ou poucos níveis.

- Sensibilidade a outliers e ruído: Random Forest é mais robusto a outliers e ruído nos dados. AdaBoost é mais sensível a outliers, pois aumenta progressivamente o peso das amostras mal classificadas.

- Mecanismo de votação: Random Forest utiliza votação com peso igual para todas as árvores na previsão final. AdaBoost implementa uma votação ponderada, na qual árvores com melhor desempenho recebem maior influência na decisão final.

## Exemplo do AdaBoost:

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
from sklearn.model_selection import cross_val_score

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

base_estimator = DecisionTreeClassifier(max_depth=1)
ada_clf = AdaBoostClassifier(estimator=base_estimator, algorithm='SAMME', random_state=42)

scores = cross_val_score(ada_clf, X, y, cv=5)
print(scores.mean())


0.9533333333333334


## Principais Hyperparametros do AdaBoost

- n_estimators: Define o número máximo de estimadores (classificadores fracos) que serão usados na construção do modelo. Um número maior pode melhorar o desempenho até certo ponto, mas aumenta o tempo de processamento e pode levar a overfitting.

- learning_rate: Também conhecido como "shrinkage", controla a contribuição de cada classificador fraco no ensemble final. Valores menores (como 0.01 ou 0.1) geralmente requerem mais estimadores mas podem oferecer melhor generalização.

- algorithm: Determina o algoritmo de boosting usado. No scikit-learn, as opções são 'SAMME' e 'SAMME.R' (embora SAMME.R esteja sendo depreciado). SAMME é o algoritmo AdaBoost original para classificação multiclasse, enquanto SAMME.R usa probabilidades de classe.

- base_estimator: Define o tipo de classificador fraco utilizado. Embora o padrão seja uma árvore de decisão com profundidade = 1 (decision stump), outros classificadores podem ser escolhidos.

- base_estimator__max_depth: Quando se usa árvores de decisão como estimadores base, a profundidade máxima de cada árvore é crucial. Árvores mais profundas capturam relações mais complexas, mas são mais propensas a overfitting.

### Utilizando GridSearch para encontrar os melhores hyperparametros para o conjunto de dados do exemplo (load_iris)

In [19]:
param_grid = {
    'estimator__max_depth': [1, 2, 3],
    'n_estimators': [30, 50, 70],
    'learning_rate': [0.01, 0.1, 1.0],
}

grid_search = GridSearchCV(
    ada_clf,
    param_grid,
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

print(grid_search.best_score_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
{'estimator__max_depth': 2, 'learning_rate': 0.1, 'n_estimators': 30}
0.9428571428571428


In [21]:
melhor_model = grid_search.best_estimator_

y_pred = melhor_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

1.0
